<a href="https://colab.research.google.com/github/dariakadina/satprak/blob/main/hw1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Задача 1

In [3]:
res= ()
for n in range(1, 12346):
    square = n ** 2
    if square % 3 == 0 and square % 4 == 0 and square % 8 != 0 and square <= 12345:
        res += (n,)
print(res)

(6, 18, 30, 42, 54, 66, 78, 90, 102)


#Задача 2

In [6]:
def rotate(m, d):
    if d == 'left':
        res = [list(row) for row in zip(*m)][::-1]
    elif d == 'right':
        res = [list(row)[::-1] for row in zip(*m)]
    for row in res:
        print(*row)

In [8]:
m = [ [1, 2, 3],
    [4, 5, 6],
    [7, 8, 9]]

rotate(m, 'left')
print()
rotate(m, 'right')

3 6 9
2 5 8
1 4 7

7 4 1
8 5 2
9 6 3


#Задача 3

In [9]:
count_chars = lambda s: {ch: s.count(ch) for ch in s}

In [10]:
print(count_chars("hello, world!"))

{'h': 1, 'e': 1, 'l': 3, 'o': 2, ',': 1, ' ': 1, 'w': 1, 'r': 1, 'd': 1, '!': 1}


#Задача 4

In [12]:
from datetime import datetime


class Book:
    def __init__(self, title, author, year, isbn, copies):
        self.title = title
        self.author = author
        self.year = year
        self.isbn = isbn
        self.copies = copies

    def info(self):
        return f"{self.title} | {self.author} | {self.year} | ISBN: {self.isbn} | copies: {self.copies}"

    def change_copies(self, delta):
        self.copies += delta


class User:
    def __init__(self, name, card_number):
        self.name = name
        self.card_number = card_number
        self.borrowed_books = []  # список ISBN

    def add_book(self, isbn):
        self.borrowed_books.append(isbn)

    def remove_book(self, isbn):
        if isbn in self.borrowed_books:
            self.borrowed_books.remove(isbn)


class Library:
    def __init__(self):
        self.books = []   # список объектов Book
        self.users = []   # список объектов User
        self.history = [] # список строк

    # --- helpers ---
    def find_book_by_isbn(self, isbn):
        for b in self.books:
            if b.isbn == isbn:
                return b
        return None

    def find_user_by_card(self, card_number):
        for u in self.users:
            if u.card_number == card_number:
                return u
        return None

    # --- book management ---
    def add_book(self, book):
        self.books.append(book)

    def delete_book(self, isbn):
        book = self.find_book_by_isbn(isbn)
        if book:
            self.books.remove(book)

    def search_books(self, title=None, author=None, year=None, isbn=None):
        result = self.books
        if title is not None:
            result = [b for b in result if title.lower() in b.title.lower()]
        if author is not None:
            result = [b for b in result if author.lower() in b.author.lower()]
        if year is not None:
            result = [b for b in result if b.year == year]
        if isbn is not None:
            result = [b for b in result if b.isbn == isbn]
        return result

    # --- user management ---
    def add_user(self, user):
        self.users.append(user)

    def delete_user(self, card_number):
        user = self.find_user_by_card(card_number)
        if user:
            self.users.remove(user)

    def search_users(self, name=None, card_number=None):
        result = self.users
        if name is not None:
            result = [u for u in result if name.lower() in u.name.lower()]
        if card_number is not None:
            result = [u for u in result if u.card_number == card_number]
        return result

    # --- issuing / returning ---
    def issue_book(self, card_number, isbn):
        user = self.find_user_by_card(card_number)
        book = self.find_book_by_isbn(isbn)

        if user is None:
            print("User not found")
            return
        if book is None:
            print("Book not found")
            return
        if book.copies <= 0:
            print("No copies available")
            return

        book.change_copies(-1)
        user.add_book(isbn)

        t = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        self.history.append(f"{t} | ISSUE | {user.name} ({user.card_number}) -> {book.title} (ISBN {isbn})")

    def return_book(self, card_number, isbn):
        user = self.find_user_by_card(card_number)
        book = self.find_book_by_isbn(isbn)

        if user is None:
            print("User not found")
            return
        if book is None:
            print("Book not found")
            return
        if isbn not in user.borrowed_books:
            print("This user didn't borrow this book")
            return

        book.change_copies(+1)
        user.remove_book(isbn)

        t = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        self.history.append(f"{t} | RETURN | {user.name} ({user.card_number}) -> {book.title} (ISBN {isbn})")

    def show_history(self):
        for row in self.history:
            print(row)


# --------- demo ----------
lib = Library()

# books
lib.add_book(Book("1984", "George Orwell", 1949, "ISBN1", 2))
lib.add_book(Book("Brave New World", "Aldous Huxley", 1932, "ISBN2", 1))
lib.add_book(Book("Python Basics", "John Doe", 2020, "ISBN3", 3))

# users
lib.add_user(User("Alice", "CARD100"))
lib.add_user(User("Bob", "CARD200"))

# issue / return scenarios
lib.issue_book("CARD100", "ISBN1")
lib.issue_book("CARD100", "ISBN2")
lib.issue_book("CARD200", "ISBN2")  # copies закончились
lib.return_book("CARD100", "ISBN2")
lib.issue_book("CARD200", "ISBN2")  # теперь можно

print("\n--- History ---")
lib.show_history()

No copies available

--- History ---
2026-02-23 20:04:28 | ISSUE | Alice (CARD100) -> 1984 (ISBN ISBN1)
2026-02-23 20:04:28 | ISSUE | Alice (CARD100) -> Brave New World (ISBN ISBN2)
2026-02-23 20:04:28 | RETURN | Alice (CARD100) -> Brave New World (ISBN ISBN2)
2026-02-23 20:04:28 | ISSUE | Bob (CARD200) -> Brave New World (ISBN ISBN2)


#Задача 5

Список b изменяется, потому что при присваивании b=a копия списка не создаётся, а переменные просто начинают ссылаться на один и тот же объект в памяти. Поэтому когда выполняется a[0]=4, изменяется сам объект списка, а не переменная a. Поскольку b указывает на тот же объект, изменения становятся видны и через него

#Задача 6


Let
$$A = \sum_{i=1}^{10000} \frac{1}{i^2},\quad B=\sum_{i=10000}^{1} \frac{1}{i^2}.$$
Calculate the values of $A$ and $B$ and compare them. What do you observe? Explain why this happens. What is the best way to calculate the value of $\sum\limits_{i=1}^{10000} \dfrac{1}{i^2}$?

In [2]:
n = 10000

sum_forward = 0.0
sum_backward = 0.0

# Суммирование от 1 до n
for i in range(1, n + 1):
    sum_forward += 1 / (i ** 2)

# Суммирование от n до 1
for i in range(n, 0, -1):
    sum_backward += 1 / (i ** 2)

print("Forward sum :", sum_forward)
print("Backward sum:", sum_backward)
print("Difference   :", f"{sum_forward - sum_backward:.16f}")

print()
print("Если складывать от 1 до n:")
print("Малые числа добавляются к уже большой сумме, часть точности теряется.")

print()
print("Если складывать от n до 1:")
print("Сначала складываются маленькие числа между собой, результат получается точнее.")

Forward sum : 1.6448340718480652
Backward sum: 1.6448340718480596
Difference   : 0.0000000000000056

Если складывать от 1 до n:
Малые числа добавляются к уже большой сумме, часть точности теряется.

Если складывать от n до 1:
Сначала складываются маленькие числа между собой, результат получается точнее.
